In [1]:
import sys

sys.path.append("/Users/shinewu/codes/xiang_wu_experimental/xiang-wu-experimental")

In [2]:
import collections
import csv
import itertools
import matplotlib.pyplot as plt
import json
import os
import pathlib
import re

import pandas as pd

from python import data, utils, word_matchers, word_models

In [9]:
print(word_matchers.extract_words("aa12 Bc! 99.#9% #Net$"))

['aa', '12', 'Bc', '99.', '9%', 'Net', '$']


In [5]:
print(word_matchers.split_at_capital("buyThisSuperCoolNFT"))

['buy', 'This', 'Super', 'Cool', 'NFT']


In [8]:
print(word_matchers.tokenize("moneyTake123 #BadApple$$ 0.9% $N$"))

['money', 'take', '123', 'bad', 'apple', '$$', '0.9%', 'n', '$']


In [4]:
MIN_30D_SALES = 1

ALL_COLLECTION_SQL = f"""
SELECT
  a.ID,
  a.SLUG,
  a.NAME,
  a.DESCRIPTION,
  a.SAFELIST_REQUEST_STATUS,
  c.IDENTIFIER AS CHAIN,
  a.EXTERNAL_URL,
  a.DISCORD_URL,
  a.TWITTER_USERNAME,
  a.INSTAGRAM_USERNAME,
  b.THIRTY_DAY_SALES AS NUM_SALES_30D,
  b.THIRTY_DAY_VOLUME AS VOLUME_IN_CRYPTO_30D,
  a.BLACKLISTED,
  a.HIDDEN,
  a.IS_DELISTED,
  a.IS_CURATED,
  a.IS_NSFW
FROM
  PC_FIVETRAN_DB.PUBLIC.API_COLLECTION a
  LEFT JOIN PC_FIVETRAN_DB.PUBLIC.API_COLLECTIONSTATS b
  ON a.ID = b.COLLECTION_ID
  LEFT JOIN PC_FIVETRAN_DB.PUBLIC.API_CHAIN c
  on a.DEFAULT_CHAIN_ID = c.ID
WHERE
  a.IS_DELISTED = FALSE
  AND a.HIDDEN = FALSE
  AND a.IS_NSFW = FALSE
  AND a.BLACKLISTED = FALSE
  AND b.THIRTY_DAY_SALES >= {MIN_30D_SALES}
ORDER BY THIRTY_DAY_SALES DESC
"""

print(ALL_COLLECTION_SQL)


SELECT
  a.ID,
  a.SLUG,
  a.NAME,
  a.DESCRIPTION,
  a.SAFELIST_REQUEST_STATUS,
  c.IDENTIFIER AS CHAIN,
  a.EXTERNAL_URL,
  a.DISCORD_URL,
  a.TWITTER_USERNAME,
  a.INSTAGRAM_USERNAME,
  b.THIRTY_DAY_SALES AS NUM_SALES_30D,
  b.THIRTY_DAY_VOLUME AS VOLUME_IN_CRYPTO_30D,
  a.BLACKLISTED,
  a.HIDDEN,
  a.IS_DELISTED,
  a.IS_CURATED,
  a.IS_NSFW
FROM
  PC_FIVETRAN_DB.PUBLIC.API_COLLECTION a
  LEFT JOIN PC_FIVETRAN_DB.PUBLIC.API_COLLECTIONSTATS b
  ON a.ID = b.COLLECTION_ID
  LEFT JOIN PC_FIVETRAN_DB.PUBLIC.API_CHAIN c
  on a.DEFAULT_CHAIN_ID = c.ID
WHERE
  a.IS_DELISTED = FALSE
  AND a.HIDDEN = FALSE
  AND a.IS_NSFW = FALSE
  AND a.BLACKLISTED = FALSE
  AND b.THIRTY_DAY_SALES >= 1
ORDER BY THIRTY_DAY_SALES DESC



In [5]:
RUN_SQL = False
ALL_COLLECTION_DF_FILE = pathlib.Path(f"/Users/shinewu/tmp/all_collection_min_30d_sales_{MIN_30D_SALES}.pickle")

if RUN_SQL:
    ALL_COLLECTION_DF = data.snowflake_run_query(ALL_COLLECTION_SQL)
    ALL_COLLECTION_DF["OPENSEA_URL"] = [utils.slug_to_url(slug) for slug in ALL_COLLECTION_DF.SLUG]
    ALL_COLLECTION_DF.to_pickle(ALL_COLLECTION_DF_FILE)
else:
    ALL_COLLECTION_DF = pd.read_pickle(ALL_COLLECTION_DF_FILE)

In [6]:
ALL_COLLECTION_DF.iloc[:10]

,ID,SLUG,NAME,DESCRIPTION,SAFELIST_REQUEST_STATUS,CHAIN,EXTERNAL_URL,DISCORD_URL,TWITTER_USERNAME,INSTAGRAM_USERNAME,NUM_SALES_30D,VOLUME_IN_CRYPTO_30D,BLACKLISTED,HIDDEN,IS_DELISTED,IS_CURATED,IS_NSFW,OPENSEA_URL
0,8531189,unidentified-contract-5lnnpz07os,Unidentified contract - 5lNnPz07oS,None,not_requested,ethereum,None,None,None,None,1.000000e+22,0.000000,False,False,False,False,False,https://opensea.io/collection/unidentified-con...
1,8515777,fraktalnftcyberbrokes-v4,FraktalNFTCyberBrokes V4,None,not_requested,ethereum,None,None,None,None,1.000000e+22,0.000000,False,False,False,False,False,https://opensea.io/collection/fraktalnftcyberb...
2,13295152,shatteredeon-rawresources,Shattered Eon Raw Resources,"**Shattered Eon is an ever-expanding, ever-unf...",not_requested,ethereum,https://shatteredeon.io,https://discord.gg/ShatteredEON,None,None,5.562781e+06,0.359235,False,False,False,False,False,https://opensea.io/collection/shatteredeon-raw...
3,6061,rarible,Rarible,Create and sell digital collectibles secured w...,approved,ethereum,https://rarible.com/,None,rariblecom,None,2.821580e+05,158.792787,False,False,False,False,False,https://opensea.io/collection/rarible
4,12922940,eejiewieiwiuruiwrq,RicoWebmaster,----,not_requested,ethereum,None,None,None,None,2.999900e+04,0.000000,False,False,False,False,False,https://opensea.io/collection/eejiewieiwiuruiwrq
5,14198078,poorlandnft,PoorLandNFT,100 million Metaverse Land NFTs belonging to t...,not_requested,ethereum,https://www.poorland.io,https://discord.gg/3QHJmsWun5,None,None,2.902500e+04,0.594000,False,False,False,False,False,https://opensea.io/collection/poorlandnft
6,14919425,ainightbirds,AINightbirds,"The stunning collection of 10,000 AINightbirds...",verified,ethereum,https://nightbirds.art/,https://discord.gg/ainightbirds,None,None,2.704400e+04,5755.317120,False,False,False,False,False,https://opensea.io/collection/ainightbirds
7,15130764,goblintownwtf,goblintown.wtf,AAAAAAAUUUUUGGGHHHHH gobblins goblinns GOBLINN...,verified,ethereum,https://goblintown.wtf/,None,None,None,2.588000e+04,35829.653166,False,False,False,False,False,https://opensea.io/collection/goblintownwtf
8,15650945,elftown-wtf,Elftown.wtf,"The Dark Elves and High Elves, the two ancient...",not_requested,ethereum,http://Elftown.wtf,None,None,None,2.447200e+04,2776.194131,False,False,False,False,False,https://opensea.io/collection/elftown-wtf
9,14195734,rarelandnft,Rare Land NFT,Welcome to Rare Land.\nRare Land NFTs are the ...,approved,ethereum,http://mlabs.land,https://discord.gg/XWxPHHDWm7,None,None,2.338600e+04,1429.763984,False,False,False,False,False,https://opensea.io/collection/rarelandnft


In [7]:
def test_with(label, s):
    matcher = word_models.LABEL_AND_MATCHER[label]
    return list(word_matchers.match_all(matcher, word_matchers.tokenize(s)))

In [8]:
test_with(
    "GET_MONEY",
    "paying you no dollar and you get paid $$$ and you receive monthly payment"
)

[(6, 9)]

In [9]:
test_with("FINANCIAL_FREEDOM", "financial total freedom!")

[(0, 3)]

In [10]:
test_with("RETIRE_EARLY", "Retires pretty sure early!")

[(0, 4)]

In [11]:
test_with("SECURITY_TRADED", "stocks will be sold to him")

[(0, 4)]

In [12]:
def label_with_matcher(row):
    all_text = str(row["NAME"]) + " " + str(row["DESCRIPTION"])
    all_tokens = word_matchers.tokenize(all_text)


    labels = []
    patterns = []
    num_matches = 0
    for label, matcher in word_models.LABEL_AND_MATCHER.items():
        matches = list(word_matchers.match_all(matcher, all_tokens))
        num_matches += len(matches)
        if matches:
            labels.append(label)
            patterns.append(
                '|'.join(' '.join(all_tokens[slice(*m)]) for m in matches))
    return labels, patterns, num_matches


LABELS_WITH_MATCHER = ALL_COLLECTION_DF.apply(label_with_matcher, axis=1)

In [13]:
new_df = ALL_COLLECTION_DF.copy()
new_df['DESCRIPTION'] = [x[:50] if x is not None else "" for x in new_df.DESCRIPTION]
new_df['LABELS'] = ['|'.join(x[0]) for x in LABELS_WITH_MATCHER]
new_df['NUM_MATCHES'] = [x[2] for x in LABELS_WITH_MATCHER]
new_df['MATCHED_PATTERNS'] = ['|'.join(x[1]) for x in LABELS_WITH_MATCHER]
columns = list(new_df.columns)
new_columns = columns[:-4]
new_columns = new_columns[:6] + columns[-4:] + new_columns[6:]
print(new_columns)
new_df = new_df[new_columns]
mask = ['CASINO' in x for x in new_df.LABELS]
print(sum(mask))
new_df[mask].head()
new_df.query("NUM_MATCHES > 0").to_csv('/Users/shinewu/tmp/xiang_financially_sensitive_NFTs_20220614.csv', index=False)

['ID', 'SLUG', 'NAME', 'DESCRIPTION', 'SAFELIST_REQUEST_STATUS', 'CHAIN', 'OPENSEA_URL', 'LABELS', 'NUM_MATCHES', 'MATCHED_PATTERNS', 'EXTERNAL_URL', 'DISCORD_URL', 'TWITTER_USERNAME', 'INSTAGRAM_USERNAME', 'NUM_SALES_30D', 'VOLUME_IN_CRYPTO_30D', 'BLACKLISTED', 'HIDDEN', 'IS_DELISTED', 'IS_CURATED', 'IS_NSFW']
63


In [14]:
desc = '''


Aiyah Verse Stickman-X Genesis 465 out of 666 Genesis Stickman-X have awakened on the Ethereum ERC721 blockchain. Nobody know when the rest will do so. These Stickman-X are the origins of the vast Aiyah Verse and will allow their masters to travel between the worlds within. New worlds are uncovered as time moves forward... to explore the new or to dominate the old, their fate in the Aiyah Verse is yours to decide.

Holders of the Stickman-X Genesis will be entitled to profit-sharing in our upcoming mobile game - Crypto Warframe as well as secondary sale revenue-sharing in our upcoming ID NFTs to be used in the game


'''

print(test_with("REVENUE_SHARE", desc))

[(87, 89), (101, 103)]


In [15]:
GROUPED = collections.defaultdict(list)

num_flagged = 0
num_double = 0

for row, (label, matched, num_matched) in zip(ALL_COLLECTION_DF.itertuples(), LABELS_WITH_MATCHER):
    if label:
        num_flagged += 1
    if len(label) > 1:
        num_double += 1
    for l, m in zip(label, matched):
        GROUPED[l].append(dict(
            match=m,
            num=num_matched,
            slug=row.SLUG,
            name='' if row.NAME is None else row.NAME,
            desc='' if row.DESCRIPTION is None else row.DESCRIPTION,
        ))

print(f"positive rate = {num_flagged} / {len(ALL_COLLECTION_DF)} = {num_flagged / len(ALL_COLLECTION_DF)}")
print(f"double rate = {num_double} / {len(ALL_COLLECTION_DF)} = {num_double / len(ALL_COLLECTION_DF)}")

positive rate = 475 / 38790 = 0.012245424078370714
double rate = 45 / 38790 = 0.001160092807424594


In [16]:
for label, rows in GROUPED.items():
    print(f"\n\n\n ===== {label} BEGIN ===== \n\n\n")
    for row in rows[:20]:
        print(f"{utils.slug_to_url(row['slug'])}  ,  label: {label}")
        print(f"num_matchs={row['num']}, match={row['match']}")
        print(row["name"])
        print(row["desc"][:100])
        print("----------\n")

    print(f"\n\n\n ===== {label} END ===== \n\n\n")




 ===== PASSIVE_INCOME BEGIN ===== 



https://opensea.io/collection/fantasyapeyachtclub  ,  label: PASSIVE_INCOME
num_matchs=1, match=passive income
Fantasy Ape Yacht Club
"Fantasy Ape Yacht Club'' is a private collection of unique digital 50 NFTs stored as ERC-721 tokens
----------

https://opensea.io/collection/dopyapeyachtclub  ,  label: PASSIVE_INCOME
num_matchs=1, match=passive income
Dopy Ape Yacht Club
The most visited place in the yacht club, where the monkeys will not be bored! "Dopy Ape Yacht Club'
----------

https://opensea.io/collection/pugging-awesome  ,  label: PASSIVE_INCOME
num_matchs=2, match=diversified investment|passive income
Pugging Awesome
5555 Pixelated Pugs that grant holders access to the most diversified investment fund on the Solana 
----------

https://opensea.io/collection/degenerate-bear-syndicate-1  ,  label: PASSIVE_INCOME
num_matchs=1, match=passive income
Degenerate Bear Syndicate
Degenerate Bear Syndicate is a collection of NFTs that gives you ac

In [17]:
for label, rows in GROUPED.items():
    print(f"{label} -> {len(rows)}")

PASSIVE_INCOME -> 125
INTEREST_GOT -> 16
REVENUE_SHARE -> 42
GET_MONEY -> 57
CASINO_UNIGRAM -> 55
SECURITY_UNIGRAM -> 37
GET_PROFIT -> 66
GET_INTEREST -> 45
IS_INVESTMENT -> 33
FINANCIAL_FREEDOM -> 9
CASINO_BETTING -> 8
MONEY_GOT -> 9
TRADE_SECURITY -> 6
PROFIT_GOT -> 11
INCREASE_INVESTMENT -> 4
I_WILL_BUY -> 3
RETIRE_EARLY -> 2
